# Ukara 1.0 Challenge Track 1 (baseline model )
**Python notebook ini menjelaskan detail baseline model UKARA. Notebook ini hanya menampilkan eksperimen dengan menggunakan data training saja untuk DATA A. _Baseline performance_ dihasilkan dengan menggunakan data test yang akan di _reveal_ pada tanggal 16 September 2019.** <br>

Ada beberapa step yang dilakukan dalam eksperimen ini yaitu:

1. Preparation
2. Ekstraksi Fitur
3. Learning
4. Menyimpan hasil prediksi ke json file


In [1]:
import sys
import numpy as np
import csv
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split
np.random.seed(42)

dataA = "/home/yunita/Data/Project/essay_scoring/Tahap_2_2019/EvalAI/Data_A/data_train_A.csv" # load data
dataA_df = pd.read_csv(dataA, header=0)
dataA_df.head() 

,RES_ID,RESPONSE,LABEL
0,TRA1,intetraksi/beradaptasi terhadap lingkungan yan...,1
1,TRA2,seperti jatuhnya meteor tsunami gempa bumi,0
2,TRA3,hanya tuhan yang tahu tantangan nya itu apaan,0
3,TRA4,mereka akan sulit beradaptasi,1
4,TRA5,"Tempat tinggal, ekonomi, dan pekerjaan",1


In [2]:
dataA_df.shape[0]

268

Karena pada eksperimen ini hanya menggunakan data training, maka data dibagi menjadi 2 part yaitu training dan test

In [3]:
trainingSet, testSet = train_test_split(dataA_df, test_size=0.2, stratify=dataA_df['LABEL'], random_state=0)

## 1. Preparation
Dalam tahap preparation, kita perlu mempersiapkan kamus *stopwords* terlebih dahulu.
* Stopwords merupakan kata-kata yang biasanya tidak signifikan secara makna (non-content words) (i.e. yang, dari, ke, dia, kamu, etc). File **stopwords.txt** berisi 763 _predefined stopwords_ dalam bahasa Indonesia.

**Anda bisa mendefiniskan sendiri kamus stopword yang akan digunakan, atau memilih tidak membuang stopword**

In [4]:
stopw = set([w for w in open('/home/yunita/Data/Code/essay_scoring/stopwords.txt','r')])
print ("Jumlah stop words:", len(stopw))

Jumlah stop words: 763


## 2. Ekstraksi Fitur
Tahap berikutnya adalah ekstraksi fitur. Pada _baseline model_, digunakan **Bag of words** sebagai fitur. Untuk ekstraksi fitur, digunakan salah satu python library yaitu Scikit Learn

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer="word", ngram_range=(1,1), stop_words=stopw, lowercase=True)

train_vect = count_vect.fit_transform(trainingSet['RESPONSE'].astype(str)) #fit transform pada data train A
test_vect = count_vect.transform(testSet['RESPONSE'].astype(str)) #transform data test 


## 3. Learning dan Evaluasi
Tahap selanjutnya adalah learning dengan mengimplementasikan algoritma machine learning yaitu AdaBoost Classifier. Evaluasi dengan metric _accuracy_

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


AdaClassifier = AdaBoostClassifier() #menggunakan default parameter
AdaClassifier.fit(train_vect , trainingSet['LABEL']) #fit pada data train
Predict = AdaClassifier.predict(test_vect) #memprediksi label pada data test


In [8]:
print ("Accuracy pada test data:", accuracy_score(testSet['LABEL'], Predict))


Accuracy pada test data: 0.759259259259


In [10]:
print ("Precision baseline pada test data:", precision_score(testSet['LABEL'], Predict))

Precision baseline pada test data: 0.878787878788


In [11]:
print ("Recall baseline pada test data:", recall_score(testSet['LABEL'], Predict))

Recall baseline pada test data: 0.763157894737


In [12]:
print ("F1 baseline pada test data:", f1_score(testSet['LABEL'], Predict))

F1 baseline pada test data: 0.816901408451


In [13]:
tn_dev, fp_dev, fn_dev, tp_dev = confusion_matrix(testSet['LABEL'], Predict).ravel()
TPR_dev = tp_dev/(tp_dev+fn_dev)
FPR_dev = fp_dev/(fp_dev+tn_dev)
print("TPR baseline:", TPR_dev, "dan FRP baseline:", FPR_dev)

TPR baseline: 0.763157894737 dan FRP baseline: 0.25


## 4. Menyimpan hasil prediksi ke json file

In [14]:
from pandas import DataFrame

prediksi_dataA = {'RES_ID': testSet['RES_ID'],
                'LABEL': Predict
                }
df_dataA = DataFrame(prediksi_dataA, columns= ['RES_ID', 'LABEL'])
df_dataA.to_json ('/home/yunita/Data/Code/essay_scoring/train/predictions_test.json', orient='records')